<img width="10%" alt="LinkedIn" src="https://media.licdn.com/dms/image/C560BAQHaVYd13rRz3A/company-logo_200_200/0/1638831589865?e=1705536000&v=beta&t=QZW5sOogDv7AXveQCCSDe3OhfDBkBFdN1VS6PJLgj74">

# LinkedIn - Get profile posts

**Tags:** #linkedin #profile #posts #command

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-10-25 (Created: 2023-10-25)

**Description:** This notebook fetches your profile's post statistics from LinkedIn and stores them in a CSV file.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [1]:
from naas_drivers import linkedin
import naas
import os
from tabulate import tabulate

### Setup variables
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `linkedin_url`: This variable represents the LinkedIn profile URL.
- `limit`: number of posts to be retrieved. Limit max is set to 15.
- `body`: This variable stores the body to be send by the webhook.

In [2]:
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
linkedin_url = "https://www.linkedin.com/in/xxxxx/"  # EXAMPLE "https://www.linkedin.com/in/XXXXXX/"
limit = 5
# output_file = "/home/ftp/abi/outputs/content-engine/"
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [3]:
# Parameters
if len(body) > 0:
    linkedin_url = body.get("linkedin_url")
    limit = body.get("limit")

## Model

### Get posts

In [ ]:
def get_linkedin_post(
    li_at,
    JSESSIONID,
    linkedin_url,
    limit,
    file_path=None,
):
    # Init
    status = "ok"
    message = ""
    limit = int(limit)
    if limit > 15:
        limit = 15
    try:
        # Get posts
        df = linkedin.connect(li_at, JSESSIONID).profile.get_posts_feed(linkedin_url, limit=limit, sleep=False)
        
        # Save posts to csv
        if not file_path:
            file_path = "LINKEDIN_POST.csv"
        df.to_csv(file_path, index=False)
        url = naas.asset.add(file_path)
        os.remove(file_path)
        
        # Clean and prep markdown
        columns = [
            "AUTHOR_NAME",
            "TEXT",
            "VIEWS",
            "COMMENTS",
            "LIKES",
            "SHARES",
            "ENGAGEMENT_SCORE",
            "POST_URL"
        ]
        df = df[columns]
        
        # Convert DataFrame to Markdown
        markdown_table = tabulate(df, headers='keys', tablefmt='pipe')
        
        # Create message
        message = f"""
        Analyze which topic/keyword has the most traction
        - Data: {markdown_table}
        - Link to download CSV: {url}
        """
    except Exception as e:
            message = f"Template error: {e}"
    return status, message

status, message = get_linkedin_post(li_at, JSESSIONID, linkedin_url, limit)

## Output

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)